## Improving GPR Metrics via Kernel Tuning

The first large chunk contains the function to train the ML model for plotting.

Beneath it, is the step-by-step workflow to achieve gpr training. The code in this notebook is a combination of material found in `gauss_reg.ipynb`, `gpr_kernel_exploration.ipynb`, and `gpr_plot_fun.ipynb`

Authors: Sofia Ingersoll## Expanding GPR Workflow for Plotting

The first large chunk contains the function to train the ML model for plotting.

Beneath it, is the step-by-step workflow to achieve gpr training. The code in this notebook is a combination of material found in `gauss_reg.ipynb`, `gpr_kernel_exploration.ipynb`, and `gpr_plot_fun.ipynb`

Authors: Sofia Ingersoll

#### Set up import libraries & data pre-processing functions from utils.py

In [26]:
# import libraries & data pre-processing functions from utils.py
import xarray as xr
import os

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import ConstantKernel, RBF, WhiteKernel
from sklearn.gaussian_process.kernels import WhiteKernel, RBF, Matern, ConstantKernel, RationalQuadratic, ExpSineSquared
from sklearn.metrics.pairwise import linear_kernel as Linear
from sklearn.metrics.pairwise import polynomial_kernel as Polynomial
from sklearn.preprocessing import MinMaxScaler 
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import accuracy_score
from sklearn.metrics import r2_score
from scipy.stats import norm


from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.svm import SVR

from ml_utils import *

In [157]:
# Request an additional 100 cores of power for processing from the server
client = get_cluster("UCSB0021", cores = 300)

# apply peer2peer network communication across multiple devices
client.cluster

/glade/u/home/singersoll/.conda/envs/meds-py/lib/python3.11/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 41259 instead
  warnings.warn(


PBSCluster(22133248, 'tcp://128.117.208.106:42813', workers=2, threads=2, memory=18.62 GiB)

#### Read & Wrangle

In [3]:
def dashboard_wrangling(param, var):
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    #----        Ifelse Load Data       ----
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # THIS FUNCTION IS TO BE ADDED SUNDAY 4/1/24
    # Heather's ifelse statement for reading & wrangling



    
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    #----            Parameter Data.          ----
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # NEED TO ADD NAME ATTRIBUTE IN WRANGLING PORTION
    param = param_wrangling(param)
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # ----    Subset User Selection Funct     ----
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    var = subset_var_cluster(var)

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # ----      Subset Var Wrangle Funct      ----
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # NEED TO ADD NAME ATTRIBUTE IN WRANGLING PORTION
    var = wrangle_var_cluster(var)
        

    return param, var

In [4]:
param, var = dashboard_wrangling('leafcn','LNC')

#### Kernel Testing

In [29]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# ----    kernel specifications     ----  
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# defining kernels
linear = ConstantKernel(constant_value=3.0, constant_value_bounds=(1e-2, 1e4)) \
         * Linear

poly = ConstantKernel(constant_value= 3, constant_value_bounds=(1e-2, 1e4)) \
        * Polynomial 

# RBF kernel in GPflow is equivalent to scikit-learn's RBF kernel
RBF = RBF(length_scale=1,length_scale_bounds=(1e-4, 1e8))

# Matern32 in GPflow is equivalent to scikit-learn's Matern with nu=1.5
Matern32 = ConstantKernel(constant_value= 3, constant_value_bounds=(1e-2, 1e4)) * Matern(length_scale=1, nu=1.5)

# Matern52 in GPflow is equivalent to scikit-learn's Matern with nu=2.5
Matern52 = ConstantKernel(constant_value= 3, constant_value_bounds=(1e-2, 1e4)) * Matern(length_scale=1, nu=2.5)

# Bias kernel in GPflow is equivalent to scikit-learn's WhiteKernel
bias = WhiteKernel(noise_level= 3, noise_level_bounds=(1e-2, 1e4))

In [44]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# ----      Split Data 90/10        ----
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# data for splitting
X_train, X_test, y_train, y_test = train_test_split(param,
                                                    var,
                                                    test_size=0.2,
                                                   # setting a seed
                                                    random_state=0)

In [16]:
X_train.shape

(400, 32)

In [17]:
X_test.shape

(100, 32)

In [18]:
y_train.shape

(400,)

In [19]:
y_test.shape

(100,)

In [21]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# ----    Kernel Specs No Tuning    ----
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# Define kernels
#linear = ConstantKernel(constant_value=3.0, constant_value_bounds=(1e-2, 1e4)) * RBF(length_scale=1, length_scale_bounds=(1e-4, 1e8))
#poly = ConstantKernel(constant_value=3, constant_value_bounds=(1e-2, 1e4)) * Polynomial
#RBF_kernel = RBF(length_scale=1, length_scale_bounds=(1e-4, 1e8))
#bias = WhiteKernel(noise_level=3, noise_level_bounds=(1e-2, 1e4))

# Combine kernels
#kernel1 = Sum(linear, bias)

In [144]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# ----    Kernel Specs No Tuning    ----
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# Define the kernel 
# r^2 -0.007
#kernel = 1.0 * RBF(length_scale=1.0, length_scale_bounds=(1e-4, 1e8)) + WhiteKernel(noise_level=3.0, noise_level_bounds=(1e-4, 1e4)) + Matern(length_scale=1.0, nu=1.5)

# Define the kernel with suitable hyperparameters
# r^2 -0.005
#kernel = 1.0 * RBF(length_scale=1.0, length_scale_bounds=(1e-4, 1e8)) \
        # + WhiteKernel(noise_level=3, noise_level_bounds=(1e-4, 1e4))

#
# Define the kernel with suitable hyperparameters
# POLY GIVING SO MUCH TROUBLE
#kernel = (ConstantKernel(3.0, constant_value_bounds=(1e-4, 1e8)) * RBF(length_scale=1.0, length_scale_bounds=(1e-4, 1e8)) +
 #         ConstantKernel(3.0, constant_value_bounds=(1e-4, 1e8)) * Matern(length_scale=1.0, nu=1.5) +
  #        ConstantKernel(3.0, constant_value_bounds=(1e-4, 1e8)) * RationalQuadratic(length_scale=1.0, alpha=0.1) +
   #       ConstantKernel(3.0, constant_value_bounds=(1e-4, 1e8)) * ExpSineSquared(length_scale=1.0, periodicity=1.0) +
    #      ConstantKernel(3.0, constant_value_bounds=(1e-4, 1e8)) * Polynomial(degree=2, X=X_train.reshape(-1,1).flatten()) + # JUP IS SUPER UNHAPPY WITH ME
     #     WhiteKernel(noise_level=3))

# Define the kernel with appropriate hyperparameters
# takes too long for a dashboard - inspired 100 core request
# R^2: -0.02
#kernel = ConstantKernel(3.0, constant_value_bounds=(1e-4, 1e8)) * RBF(length_scale=1.0, length_scale_bounds=(1e-4, 1e8)) \
 #        + ConstantKernel(3.0, constant_value_bounds=(1e-4, 1e8)) * Matern(length_scale=1.0, nu=1.5) \
  #       + ConstantKernel(3.0, constant_value_bounds=(1e-4, 1e8)) * RationalQuadratic(length_scale=1.0, alpha=0.1) \
   #      + ConstantKernel(3.0, constant_value_bounds=(1e-4, 1e8)) * ExpSineSquared(length_scale=1.0, periodicity=1.0) \
    #     + WhiteKernel(noise_level=3)


# Define the kernel with suitable hyperparameters
# R^2: -0.007
#kernel = ConstantKernel(3.0, constant_value_bounds=(1e-4, 1e8)) * RBF(length_scale=1.0) \
 #        + WhiteKernel(noise_level=3.0, noise_level_bounds=(1e-4, 1e8)) \
  #       + ConstantKernel(3.0, constant_value_bounds=(1e-4, 1e8)) \
   #      * RationalQuadratic(length_scale=1.0, alpha=0.1) \
    #    * Matern(length_scale=1.0, nu=1.5)

# Define the kernel with suitable hyperparameters
# R^2:-0.007
#kernel = ConstantKernel(1.0, constant_value_bounds=(1e-4, 1e8)) * RBF(length_scale=1.0, length_scale_bounds=(1e-4, 1e8)) \
   #      + WhiteKernel(noise_level=1.0, noise_level_bounds=(1e-4, 1e8)) \
    #     + ConstantKernel(1.0, constant_value_bounds=(1e-4, 1e8)) \
     #    * RationalQuadratic(length_scale=1.0, alpha=0.1) \
      #   + ConstantKernel(1.0, constant_value_bounds=(1e-4, 1e8)) \
       #  * Matern(length_scale=1.0, nu=1.5)







# Define the kernel with suitable hyperparameters
# R^2:Training R^2: 0.01
#kernel = ConstantKernel(1.0, constant_value_bounds=(1e-4, 1e4)) * RBF(length_scale=1.0, length_scale_bounds=(1e-4, 1e8)) \
 #        + ConstantKernel(1.0, constant_value_bounds=(1e-4, 1e4)) * Matern(length_scale=1.0, nu=1.5) \
  #       + ConstantKernel(1.0, constant_value_bounds=(1e-4, 1e4)) * RationalQuadratic(length_scale=1.0, alpha=0.1) \
   #      + ConstantKernel(1.0, constant_value_bounds=(1e-4, 1e4)) * ExpSineSquared(length_scale=1.0, periodicity=1.0) \
    #     + WhiteKernel(noise_level=1.0, noise_level_bounds=(1e-4, 1e4))


# Define the kernel with suitable hyperparameters
# R^2:Training R^2: 0.01
kernel = ConstantKernel(3.0, constant_value_bounds=(1e-6, 1e4)) * RBF(length_scale=1.0, length_scale_bounds=(1e-4, 1e8)) \
         + ConstantKernel(3.0, constant_value_bounds=(1e-6, 1e4)) * Matern(length_scale=1.0, nu=1.5) \
         + ConstantKernel(3.0, constant_value_bounds=(1e-6, 1e4)) * RationalQuadratic(length_scale=1.0, alpha=0.1) \
         + ConstantKernel(3.0, constant_value_bounds=(1e-6, 1e4)) * ExpSineSquared(length_scale=1.0, periodicity=1.0) \
         + WhiteKernel(noise_level=3.0, noise_level_bounds=(1e-6, 1e4))







# Define the kernel with suitable hyperparameters
# R^2:-0.007
#kernel = ConstantKernel(1.0, constant_value_bounds=(1e-6, 1e8)) * RBF(length_scale=1.0, length_scale_bounds=(1e-6, 1e8)) \
   #      + ConstantKernel(1.0, constant_value_bounds=(1e-6, 1e8)) * Matern(length_scale=1.0, nu=1.5) \
    #     + ConstantKernel(1.0, constant_value_bounds=(1e-6, 1e8)) * RationalQuadratic(length_scale=1.0, alpha=0.1) \
     #    + ConstantKernel(1.0, constant_value_bounds=(1e-6, 1e8)) * ExpSineSquared(length_scale=1.0, periodicity=1.0) \
      #   + WhiteKernel(noise_level=3.0, noise_level_bounds=(1e-6, 1e8))


In [123]:
kernel

1.73**2 * RBF(length_scale=1) + WhiteKernel(noise_level=3) + 1.73**2 * RationalQuadratic(alpha=0.1, length_scale=1) + 1.73**2 * Matern(length_scale=1, nu=1.5)

In [145]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# ----        GPR Model Recipe      ----
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
gp_model = GaussianProcessRegressor(kernel=kernel,
                                    n_restarts_optimizer=20,
                                    random_state=42,
                                    normalize_y = True)

# Scale the normalized target variable y to the desired range
scaler = MinMaxScaler(feature_range=(20, 40))
y_train_scaled = scaler.fit_transform(y_train.values.reshape(-1, 1)).flatten()

In [125]:
gp_model

GaussianProcessRegressor(kernel=1.73**2 * RBF(length_scale=1) + WhiteKernel(noise_level=3) + 1.73**2 * RationalQuadratic(alpha=0.1, length_scale=1) + 1.73**2 * Matern(length_scale=1, nu=1.5),
                         n_restarts_optimizer=20, normalize_y=True,
                         random_state=42)

In [146]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# ----         Fit the Model        ----
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# Fit the model to the training data 
gp_model = gp_model.fit(X_train, y_train_scaled)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# ----         Get Predictions      ----
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# Make predictions
y_pred, y_std = gp_model.predict(X_test, return_std=True)

# Inverse transform the scaled predictions to the original scale
y_pred = scaler.inverse_transform(y_pred.reshape(-1, 1)).flatten()

/glade/u/home/singersoll/.conda/envs/meds-py/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:445: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__k1__k1__k2__k1__constant_value is close to the specified lower bound 0.0001. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/glade/u/home/singersoll/.conda/envs/meds-py/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:445: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__k1__k2__k1__constant_value is close to the specified lower bound 0.0001. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(


In [17]:
gp_model

GaussianProcessRegressor(kernel=1**2 * RBF(length_scale=1) + WhiteKernel(noise_level=1) + Matern(length_scale=1, nu=1.5),
                         n_restarts_optimizer=20, normalize_y=True,
                         random_state=42)

In [147]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# ----         Collect Metrics      ----
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# Verify training score
train_score = gp_model.score(X_train, y_train_scaled)
    
# Calculate Mean Absolute Error
mae = mean_absolute_error(y_test, y_pred)

# Calculate R^2
r2_train = r2_score(y_test, y_pred)
# Calculate RMSE
rmse_train = np.sqrt(mean_squared_error(y_test, y_pred))

# Create a DataFrame to store the results for plotting
results_df = pd.DataFrame({
     'y_pred': y_pred,
    'y_std': y_std,
    'y_test': y_test,
     'X_test': [x.tolist() for x in X_test],  # Convert array to list for DataFrame
})

# Add metrics to the DataFrame
results_df['R^2'] = r2_train
results_df['RMSE'] = rmse_train
results_df['Mean Absolute Error'] = mae
    
# Print Training Metrics
print("Training R^2:", r2_train)
print("Training RMSE:", rmse_train)
print("Mean Absolute Error:", mae)
print("Training Score:", train_score)

Training R^2: 0.00956188389566448
Training RMSE: 0.19239177385998105
Mean Absolute Error: 0.15591733241963002
Training Score: 0.8614164816798965


#### Linnia's Kernel Recipe, not specs tho

R-squared value (Combined Tree Kernel 1): -0.006963103407625626


R-squared value (Combined Grass Kernel 3): -0.006963099087646141



In [150]:
from sklearn.gaussian_process.kernels import ConstantKernel, RBF, Matern, RationalQuadratic, ExpSineSquared, WhiteKernel, Sum, Product

# Define individual kernels
linear_kernel = ConstantKernel() * RBF()
rbf_kernel = ConstantKernel() * RBF()
poly_kernel = ConstantKernel() * RationalQuadratic()
matern_kernel = ConstantKernel() * Matern()
noise_kernel = WhiteKernel()


# Combine kernels in various ways
# Option 1: Linnia Grass kernel, Sum of kernels
# Combine kernels using Sum and Product operations
combined_kernel_1 = Sum(linear_kernel, rbf_kernel)
combined_kernel_1 = Sum(combined_kernel_1, poly_kernel)
combined_kernel_1 = Sum(combined_kernel_1, matern_kernel)
combined_kernel_1 = Sum(combined_kernel_1, noise_kernel)

# Option 2: Product of kernels
combined_kernel_2 = Product(linear_kernel, rbf_kernel)
#combined_kernel_2 = Product(combined_kernel_2, poly_kernel)
#combined_kernel_2 = Product(combined_kernel_2, matern_kernel)
#combined_kernel_2 = Product(combined_kernel_2, noise_kernel)

# Option 3: Linnia Tree kernel
combined_kernel_3 = Sum(combined_kernel_2, matern_kernel)
combined_kernel_3 = Sum(combined_kernel_3, noise_kernel)


# Initialize Gaussian Process Regressors with the custom kernels
gpr_model_1 = GaussianProcessRegressor(kernel=combined_kernel_1, n_restarts_optimizer=20, random_state=42)
#gpr_model_2 = GaussianProcessRegressor(kernel=combined_kernel_2, n_restarts_optimizer=20, random_state=42)
gpr_model_3 = GaussianProcessRegressor(kernel=combined_kernel_3, n_restarts_optimizer=20, random_state=42)

# Fit the models to the training data
gpr_model_1.fit(X_train, y_train)
gpr_model_3.fit(X_train, y_train)

# Predict on the test set
y_pred_1 = gpr_model_1.predict(X_test)
y_pred_3 = gpr_model_3.predict(X_test)

# Calculate R-squared values
r2_1 = r2_score(y_test, y_pred_1)
r2_3 = r2_score(y_test, y_pred_3)

# Print the R-squared values
print("R-squared value (Combined Kernel 1):", r2_1)
print("R-squared value (Combined Kernel 3):", r2_3)

/glade/u/home/singersoll/.conda/envs/meds-py/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:445: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__k1__k1__k1__k1__constant_value is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/glade/u/home/singersoll/.conda/envs/meds-py/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:455: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__k1__k1__k1__k2__length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/glade/u/home/singersoll/.conda/envs/meds-py/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:455: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__k1__k1__k2__k2__length_scale is close to the specified upper bound 100000.0. Increasing the bound and

R-squared value (Combined Kernel 1): -0.006963103407625626
R-squared value (Combined Kernel 3): -0.006963099087646141


#### Hyperparameter Tuning Approach

In [ ]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid
param_grid = {
    'kernel': [linear_kernel, rbf_kernel, matern_kernel, poly_kernel, noise_kernel],
    'alpha': [1e-6, 1e-5, 1e-4, 1e-3],  # Example values for alpha (adjust as needed)
}

# Perform grid search with cross-validation
grid_search = GridSearchCV(GaussianProcessRegressor(), param_grid=param_grid, cv=5, scoring='r2')
grid_search.fit(X_train, y_train)

# Get the best hyperparameters and model
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

# Predict on the test set using the best model
y_pred = best_model.predict(X_test)

# Evaluate the best model
r2_score = best_model.score(y_test, y_pred)
print("Best R^2 Score:", r2_score)
print("Best Hyperparameters:", best_params)

#### Combo Approach

In [149]:
# Define the individual kernels
#linear_kernel = ConstantKernel(1.0, constant_value_bounds=(1e-6, 1e8)) * RBF(length_scale=1.0, length_scale_bounds=(1e-6, 1e8))
#rbf_kernel = ConstantKernel(1.0, constant_value_bounds=(1e-6, 1e8)) * Matern(length_scale=1.0, nu=1.5)
#poly_kernel = ConstantKernel(1.0, constant_value_bounds=(1e-6, 1e8)) * RationalQuadratic(length_scale=1.0, alpha=0.1)
#matern_kernel = ConstantKernel(1.0, constant_value_bounds=(1e-6, 1e8)) * ExpSineSquared(length_scale=1.0, periodicity=1.0)
#noise_kernel = WhiteKernel(noise_level=3.0, noise_level_bounds=(1e-6, 1e8))

# Combine kernels in various ways
# Option 1: Sum of kernels
# Combine kernels using Sum and Product operations
#combined_kernel_1 = Sum(linear_kernel, rbf_kernel)
#combined_kernel_1 = Sum(combined_kernel_1, poly_kernel)
#combined_kernel_1 = Sum(combined_kernel_1, matern_kernel)
#combined_kernel_1 = Sum(combined_kernel_1, noise_kernel)

# Option 2: Product of kernels
#combined_kernel_2 = Product(linear_kernel, rbf_kernel)
#combined_kernel_2 = Product(combined_kernel_2, poly_kernel)
#combined_kernel_2 = Product(combined_kernel_2, matern_kernel)
#combined_kernel_2 = Product(combined_kernel_2, noise_kernel)

# Initialize Gaussian Process Regressors with the custom kernels
#gpr_model_1 = GaussianProcessRegressor(kernel=combined_kernel_1, n_restarts_optimizer=20, random_state=42)
#gpr_model_2 = GaussianProcessRegressor(kernel=combined_kernel_2, n_restarts_optimizer=20, random_state=42)

# Fit the models to the training data
#gpr_model_1.fit(X_train, y_train)
#gpr_model_2.fit(X_train, y_train)

# Predict on the test set
#y_pred_1 = gpr_model_1.predict(X_test)
#y_pred_2 = gpr_model_2.predict(X_test)

# Calculate R-squared values
#r2_1 = r2_score(y_test, y_pred_1)
#r2_2 = r2_score(y_test, y_pred_2)

# Print the R-squared values
#print("R-squared value (Combined Kernel 1):", r2_1)
#print("R-squared value (Combined Kernel 2):", r2_2)

R-squared value (Combined Kernel 1): -0.007432064689027351
R-squared value (Combined Kernel 2): -20.13405405817297


#### SVR kernel fit for gpr kernel optimization

Honestly, svr kernel takes so long to fit

Support Vector Regression (SVR), is specifically designed for predicting continuous values.

In [121]:
# takes extremely long, need a lot of power
# Define SVR with RBF kernel
#svr_rbf = SVR(kernel='rbf')

# Fit SVR model
#svr_rbf.fit(X_train, y_train)

In [ ]:
# Use RBF kernel with Gaussian Process Regressor
#gpr_model = GaussianProcessRegressor(kernel=svr_rbf,
 #                                    n_restarts_optimizer=20,
  #                                   random_state=42,
   #                                  normalize_y=True)

# Fit GPR model
#gpr_model.fit(X_train, y_train)

# Predict on the test set
#y_pred_gpr = gpr_model.predict(X_test)

# Calculate R-squared values
#r2_gpr = r2_score(y_test, y_pred_gpr)

# Print R-squared values
#print("R-squared value (GPR RBF Kernel):", r2_gpr)

In [45]:
# Define SVR with different kernels
#svr_linear = SVR(kernel='linear')
#svr_poly = SVR(kernel='poly', degree=2)  # Adjust degree as needed
#svr_rbf = SVR(kernel='rbf')

# Fit SVR models
#svr_linear.fit(X_train, y_train)
#svr_poly.fit(X_train, y_train)
#svr_rbf.fit(X_train, y_train)

# Predict on the test set
#y_pred_linear = svr_linear.predict(X_test)
#y_pred_poly = svr_poly.predict(X_test)
#y_pred_rbf = svr_rbf.predict(X_test)

# Calculate R-squared values
#r2_linear = r2_score(y_test, y_pred_linear)
#r2_poly = r2_score(y_test, y_pred_poly)
#r2_rbf = r2_score(y_test, y_pred_rbf)

# Print R-squared values
#print("R-squared value (Linear Kernel):", r2_linear)      # r^2: -0.1
#print("R-squared value (Polynomial Kernel):", r2_poly)    # r^2: -0.4
#print("R-squared value (RBF Kernel):", r2_rbf)            # r^2: 0.0004

R-squared value (Linear Kernel): -0.11855674384558146
R-squared value (Polynomial Kernel): -0.4091913008482797
R-squared value (RBF Kernel): 0.00035222430786163716


In [46]:
# r^2: -0.2
# Initialize SVR model with RBF kernel and adjust hyperparameters
#svr_rbf = SVR(kernel='rbf', C=100, gamma=0.1)  # Adjust C and gamma values as needed

# Fit SVR model
#svr_rbf.fit(X_train, y_train)

# Predict on the test set
#y_pred_rbf = svr_rbf.predict(X_test)

# Calculate R-squared value
#r2_rbf = r2_score(y_test, y_pred_rbf)

# Print R-squared value
#print("R-squared value (RBF Kernel):", r2_rbf)

R-squared value (RBF Kernel): -0.1761281392496128


In [48]:
# r^2: -0.5
# Initialize SVR model with RBF kernel and adjust hyperparameters
#svr_rbf = SVR(kernel='rbf', C=1000, gamma=0.01, epsilon=0.1)  # Adjust C, gamma, and epsilon values as needed

# Fit SVR model
#svr_rbf.fit(X_train, y_train)

# Predict on the test set
#y_pred_rbf = svr_rbf.predict(X_test)

# Calculate R-squared value
#r2_rbf = r2_score(y_test, y_pred_rbf)

# Print R-squared value
#print("R-squared value (RBF Kernel):", r2_rbf)

R-squared value (RBF Kernel): -0.46781817547811433


#### Custom kernel

In [89]:
y_train.shape
X_train.shape

(400, 32)

In [59]:
# r^2: -0.1
# Define custom kernel with suitable hyperparameters
#def custom_kernel(X, Y):
 #   linear_kernel = np.dot(X, Y.T)
 #   rbf_kernel = np.exp(-0.01 * np.sum((X[:, np.newaxis] - Y[np.newaxis, :]) ** 2, axis=2))
  #  return 0.8 * linear_kernel + 0.2 * rbf_kernel

# Initialize SVR model with custom kernel
#svr_custom = SVR(kernel=custom_kernel)

# Fit SVR model
#svr_custom.fit(X_train, y_train)

# Predict on the test set
#y_pred_custom = svr_custom.predict(X_test)

# Calculate R-squared value
#r2_custom = r2_score(y_test, y_pred_custom)

# Print R-squared value
#print("R-squared value (Custom Kernel):", r2_custom)

R-squared value (Custom Kernel): -0.11824107373804238


In [66]:
# r^2: -0.1
# Define custom kernel with suitable hyperparameters
#def custom_kernel(X, Y):
  #  linear_kernel = np.dot(X, Y.T)
   # rbf_kernel = np.exp(-0.005 * np.sum((X[:, np.newaxis] - Y[np.newaxis, :]) ** 2, axis=2))
  #  return 0.9 * linear_kernel + 0.1 * rbf_kernel

# Initialize SVR model with custom kernel
#svr_custom = SVR(kernel=custom_kernel)

# Fit SVR model
#svr_custom.fit(X_train, y_train)

# Predict on the test set
#y_pred_custom = svr_custom.predict(X_test)

# Calculate R-squared value
#r2_custom = r2_score(y_test, y_pred_custom)

# Print R-squared value
#print("R-squared value (Custom Kernel):", r2_custom)


R-squared value (Custom Kernel): -0.11811799142412172


In [55]:
# r^2: -0.4
# Define custom kernel with suitable hyperparameters
#def custom_kernel(X, Y):
 #   linear_kernel = np.dot(X, Y.T)
  #  rbf_kernel = np.exp(-0.001 * np.sum((X[:, np.newaxis] - Y[np.newaxis, :]) ** 2, axis=2))
   # polynomial_kernel = (1 + np.dot(X, Y.T)) ** 2  # 2nd-degree polynomial kernel
   # return 0.7 * linear_kernel + 0.2 * rbf_kernel + 0.1 * polynomial_kernel

# Initialize SVR model with custom kernel
#svr_custom = SVR(kernel=custom_kernel)

# Fit SVR model
#svr_custom.fit(X_train, y_train)

# Predict on the test set
#y_pred_custom = svr_custom.predict(X_test)

# Calculate R-squared value
#r2_custom = r2_score(y_test, y_pred_custom)

# Print R-squared value
#print("R-squared value (Custom Kernel):", r2_custom)

R-squared value (Custom Kernel): -0.3826302901013783


In [67]:
# r^2: -0.2
#def custom_kernel(X, Y):
 #   linear_kernel = np.dot(X, Y.T)
  #  polynomial_kernel = (1 + np.dot(X, Y.T)) ** 3  # 3rd-degree polynomial kernel
   # return 0.9 * linear_kernel + 0.1 * polynomial_kernel

# Initialize SVR model with custom kernel
#svr_custom = SVR(kernel=custom_kernel)

# Fit SVR model
#svr_custom.fit(X_train, y_train)

# Predict on the test set
#y_pred_custom = svr_custom.predict(X_test)

# Calculate R-squared value
#r2_custom = r2_score(y_test, y_pred_custom)

# Print R-squared value
#print("R-squared value (Custom Kernel):", r2_custom)

R-squared value (Custom Kernel): -0.21612689667235996


# Function

In [12]:
def train_emulator(param, var):

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # ----      Split Data 90/10        ----
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # data for splitting
    X_train, X_test, y_train, y_test = train_test_split(param,
                                                        var,
                                                        test_size=0.2,
                                                        # setting a seed
                                                        random_state=0)

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # ----    Kernel Specs No Tuning    ----
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # initiate the model without tuning
    kernel = Linear * RBF(length_scale=1, 
                        length_scale_bounds=(1e-4, 1e8))

    gp_model = GaussianProcessRegressor(kernel=kernel,
                                        n_restarts_optimizer=20,
                                        random_state=42,
                                        normalize_y = True)
    
    # Scale the normalized target variable y to the desired range
    scaler = MinMaxScaler(feature_range=(20, 40))
    y_train_scaled = scaler.fit_transform(y_train.values.reshape(-1, 1)).flatten()
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # ----         Fit the Model        ----
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # Fit the model to the training data
    gp_model = gp_model.fit(X_train, y_train_scaled)

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # ----         Get Predictions      ----
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # Make predictions
    y_pred, y_std = gp_model.predict(X_test, return_std=True)

    # Inverse transform the scaled predictions to the original scale
    y_pred = scaler.inverse_transform(y_pred.reshape(-1, 1)).flatten()

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # ----         Collect Metrics      ----
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # Verify training score
    train_score = gp_model.score(X_train, y_train_scaled)
    
    # Calculate Mean Absolute Error
    mae = mean_absolute_error(y_test, y_pred)

    # Calculate R^2
    r2_train = r2_score(y_test, y_pred)
    # Calculate RMSE
    rmse_train = np.sqrt(mean_squared_error(y_test, y_pred))

    # Create a DataFrame to store the results for plotting
    results_df = pd.DataFrame({
        'y_pred': y_pred,
        'y_std': y_std,
        'y_test': y_test,
        'X_test': [x.tolist() for x in X_test],  # Convert array to list for DataFrame
    })

    # Add metrics to the DataFrame
    results_df['R^2'] = r2_train
    results_df['RMSE'] = rmse_train
    results_df['Mean Absolute Error'] = mae
    
    # Print Training Metrics
    print("Training R^2:", r2_train)
    print("Training RMSE:", rmse_train)
    print("Mean Absolute Error:", mae)
    print("Training Score:", train_score)
    
    return results_df

In [13]:
emulation4 = train_emulator(param, var)

TypeError: loop of ufunc does not support argument 0 of type function which has no callable log method